In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

In [3]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [4]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [5]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

,CategoryID,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal"
5,6,Meat/Poultry,Prepared meats
6,7,Produce,Dried fruit and bean curd
7,8,Seafood,Seaweed and fish


In [6]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [7]:
tasks+=1
pd.read_sql(
    '''
    select CustomerName, Country, Address from Customers
    where Country in ('Germany', 'France') or City = 'Madrid'
    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [8]:
tasks+=1
pd.read_sql(
    '''
    select Country, cnt from
    (select CustomerName, Country, count() cnt from Customers
    group by Country
    order by cnt desc
    limit 3)

    ''',
    con,
)

,Country,cnt
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [9]:
tasks+=1
pd.read_sql(
    '''
    select s.ShipperName, o.OrderDate from
    (select * from 
    (select * from Orders
    order by OrderDate limit 10)
    order by OrderDate desc limit 1)
    o, Shippers s
    where o.ShipperID = s.ShipperID

    ''',
    con,
)

,ShipperName,OrderDate
0,Federal Shipping,02.10.1996


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [10]:
tasks+=1
pd.read_sql(
    '''
    select OrderID, ProductName, Unit, Quantity, Price from (
    select * from OrderDetails as o
    join Products as p
    on o.ProductID = p.ProductID)
    where OrderID =
    (select OrderID from (
    select OrderID, ProductName, Unit, Quantity, Price, sum(Price*Quantity) sum from (
    select * from OrderDetails as o
    join Products as p
    on o.ProductID = p.ProductID)
    group by OrderID
    order by sum desc)
    limit 1)

    ''',
    con,
)

,OrderID,ProductName,Unit,Quantity,Price
0,10372,Sir Rodney's Marmalade,30 gift boxes,12,81
1,10372,Côte de Blaye,12 - 75 cl bottles,40,263.5
2,10372,Camembert Pierrot,15 - 300 g rounds,70,34
3,10372,Mozzarella di Giovanni,24 - 200 g pkgs.,42,34.8


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [11]:
tasks+=1
pd.read_sql(
    '''
    select OrderID, ProductName, Quantity from OrderDetails as o
    join Products as p
    on o.ProductID = p.ProductID
    where ProductName = 
    (select ProductName from
    (select ProductName, sum(Quantity) sum from OrderDetails as o
    join Products as p
    on o.ProductID = p.ProductID
    group by ProductName
    order by sum desc)
    limit 1)

    ''',
    con,
)

,OrderID,ProductName,Quantity
0,10253,Gorgonzola Telino,20
1,10272,Gorgonzola Telino,40
2,10273,Gorgonzola Telino,15
3,10325,Gorgonzola Telino,4
4,10335,Gorgonzola Telino,25
5,10342,Gorgonzola Telino,56
6,10356,Gorgonzola Telino,30
7,10359,Gorgonzola Telino,70
8,10363,Gorgonzola Telino,20
9,10374,Gorgonzola Telino,30


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [12]:
tasks+=1
pd.read_sql(
    '''
    select SupplierName, Country, ContactName, Phone from Orders o
    join OrderDetails od
    on o.OrderId = od.OrderID
    join Products p
    on od.ProductID = p.ProductID
    join Suppliers sup
    on p.SupplierID = sup.SupplierID
    group by p.SupplierID
    order by count() desc
    limit 5

    ''',
    con,
)

,SupplierName,Country,ContactName,Phone
0,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755
1,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343
2,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448
3,Norske Meierier,Norway,Beate Vileid,(0)2-953010
4,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [13]:
tasks+=1
pd.read_sql(
    '''
    select Country, CategoryName, sum(Quantity*Price) sum
    from Orders o join Customers cust on o.CustomerID = cust.CustomerID
    join OrderDetails od on o.OrderID = od.OrderID
    join Products p on od.ProductID = p.ProductID
    join Categories cat on p.CategoryID = cat.CategoryID
    where cust.Country = 'Brazil'
    group by p.CategoryID
    order by sum(Quantity*Price) desc
    limit 1

    ''',
    con,
)

,Country,CategoryName,sum
0,Brazil,Beverages,13636.0


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [14]:
tasks+=1
pd.read_sql(
    '''
    select (max(oPrices.OrderPrice) - min(oPrices.OrderPrice)) difference
    from
    (select o.OrderID, sum(od.Quantity*p.Price) OrderPrice
    from Orders o join OrderDetails od on o.OrderID = od.OrderID
    join Products p on p.ProductID = od.ProductID
    join Customers cust on cust.CustomerID = o.CustomerID
    where cust.Country = 'USA'
    group by o.OrderID) oPrices

    ''',
    con,
)

,difference
0,7698.45


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [15]:
tasks+=1
pd.read_sql(
    '''
    select count() OrdersCount, (e.FirstName || ' ' || e.LastName) name
    from Orders o join Employees e on o.EmployeeID = e.EmployeeID
    group by e.EmployeeID
    order by e.BirthDate desc
    limit 3

    ''',
    con,
)

,OrdersCount,name
0,31,Janet Leverling
1,14,Robert King
2,40,Margaret Peacock


10. Сколько банок крабового мяса всего было заказано.

In [16]:
tasks+=1
pd.read_sql(
    '''
    select sum(o.Quantity * substr(Unit, 1, 2)) all_cnt_crab
    from Products p
    join OrderDetails o
    on p.ProductID = o.ProductID
    where p.ProductName = 'Boston Crab Meat'
    
    ''',
    con,
)

,all_cnt_crab
0,6144


In [17]:
if tasks==10:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
